In [ ]:
%load_ext autoreload
%autoreload 2


import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"

In [ ]:
import edesdetectrl.util.dm_env as util_dm_env
from edesdetectrl.environments.m_mode_binary_classification import (
    EDESMModeClassification_v0,
)

env = util_dm_env.GymWrapper(EDESMModeClassification_v0("echonet", "simple"))
timestep = env.reset()


In [ ]:
from edesdetectrl.agents.dqn import agent
import edesdetectrl.agents.dqn.config as dqn_config
import jax.numpy as jnp
from jax import random, jit


class MockNetwork:
    def apply(self, *args):
        return jnp.ones(8), None


policy = agent.make_inference_fn(MockNetwork(), dqn_config.DQNConfig(), False)
params, state = None, None
key = random.PRNGKey(42)
policy_state = policy.init()
inference = policy#jit(policy)
obs = env.reset().observation


In [ ]:
import gif
import matplotlib.pyplot as plt
from edesdetectrl.environments.m_mode_binary_classification.line import MModeLine


@gif.frame
def frame(env, policy_state):
    fig, ax = plt.subplots(nrows=2, figsize=(12,8))
    ax[0].imshow(env._environment.render(), aspect="auto")
    temp_mmode = MModeLine(
        policy_state.exploration_target.line,
        env._environment.mmode_line.bounds,
    )
    ax[1].imshow(temp_mmode.visualize_line(), aspect="auto")
    fig.tight_layout()
    return fig


frames = [frame(env, policy_state)]
for _ in range(20):
    key, = random.split(key, 1)
    mmode_line = env._environment.mmode_line
    action, policy_state = inference(key, policy_state, params, state, obs, mmode_line)
    timestep = env.step(action)
    obs = timestep.observation
    frames.append(frame(env, policy_state))

gif.save(frames, "temp2.gif")

In [ ]:
from edesdetectrl.environments.m_mode_binary_classification.line import Line
policy_state.exploration_target.line = Line(50,40,50,60)